In [61]:
!set PYTHONIOENCODING=utf-8

In [63]:
import pandas as pd
import json
import re
import kss
from tqdm import tqdm
from random import sample
import numpy as np
import os

## 데이터 셋 형성 (colab에서 진행)

In [65]:
data1_path = r'F:\경민서-코딩\NLP\학습데이터(기사)\017.뉴스 기사 기계독해 데이터\01.데이터\1.Training\원천데이터\TS_span_extraction.json'
data2_path = r'F:\경민서-코딩\NLP\학습데이터(기사)\017.뉴스 기사 기계독해 데이터\01.데이터\1.Training\원천데이터\TS_text_entailment.json'

In [67]:
# JSON 파일에서 탭 문자 제거하는 함수
def remove_tabs(text):
  # 탭 문자를 공백으로 교체
  return text.replace('\t', ' ')  # 공백으로 교체할 수도 있고, 그냥 제거할 수도 있음

# JSON 파일을 읽고 탭 문자 제거하기
with open(data2_path, 'r', encoding='utf-8', errors='ignore') as f:
  json_str = f.read()

# 탭 문자 제거
cleaned_json_str = remove_tabs(json_str)

# JSON 파싱
try:
  data2 = json.loads(cleaned_json_str)
  print("JSON 파싱 성공!")
except json.JSONDecodeError as e:
  print(f"에러 발생: {e}")

에러 발생: Expecting property name enclosed in double quotes: line 37203 column 1 (char 2905224)


In [45]:
cleaned_json_str[29052124:29052324]

'완화했다. 정의당은 “벌금형 가중 등 처벌이 강화돼야 기업 스스로 산재 예방에 나선다”(강 원내대표)고 반대했고, 정부는 중대재해의 개념을 지금처럼 ‘사망자 1명 이상’으로 유지할 경우 처벌 수위는 더 낮춰야 한다고 했다.\\n\\n",\n     "context_id": "1071309"\n    }\n   ]\n  },\n  {\n   "doc_id": "JAI_ar'

In [53]:
print(cleaned_json_str[2905214:2905234])

졌다. \n ",
������M�


In [7]:
# 'paragraphs'가 포함된 데이터를 평탄화
df_paragraphs = pd.json_normalize(data2['data'], 'paragraphs', ['doc_id', 'doc_title'], sep='_')

# 출력
df_paragraphs.head()

,context,context_id,doc_id,doc_title
0,2022년 대통령 선거를 진두지휘할 여당의 신임 당대표에 5선 송영길 의원이 당선됐...,579695,01100201.20210502175653001,민주당 새 당대표에 86맏형 송영길…최고위원은 ‘친문’ 강화
1,북한이 2일 대남·대미 담화 3건을 동시다발적으로 발표하면서 남측과 미국을 맹비난했...,579697,01100201.20210502172618001,하루3번 동시다발 ‘말폭탄’ 던진 北…한·미 겨냥 기선제압 나섰다
2,서울시는 코로나19 완치자의 빠른 일상 회복을 돕기 위해 4주간 건강관리서비스를 지...,579698,01100201.20210502111625001,"서울시, 코로나19 완치자 일상 회복 돕는다"
3,전남도소방본부가 전국 최초로 사회적·경제적 취약계층으로 찾아가는 ‘119생활안전전담...,579699,01100201.20210502103054001,"전남소방, 전국 첫 ‘119생활안전전담대’ 운영"
4,여성가족부가 지난달 27일 ‘제4차 건강가정기본계획(2021~2025)’을 통해 난...,579701,01100201.20210502145632001,"“비혼 출산, 생명윤리 파괴 위험성 커… 졸속 아닌 충분한 사회적 합의 선행돼야”"


In [8]:
tqdm.pandas()
context_df = df_paragraphs['context']
context_df = context_df.progress_apply(lambda x: kss.split_sentences(x))


  0%|          | 0/24011 [00:00<?, ?it/s]WARNING:root:Oh! You have mecab in your environment. Kss will take this as a backend! :D

100%|██████████| 24011/24011 [17:27<00:00, 22.93it/s]


In [9]:
context_df2 = context_df.copy()

In [10]:
row_count = 0
def select_sentence(sentence_list):
  global row_count
  sent_group = []

    # 문장 리스트가 너무 짧으면 빈 리스트 반환
  if len(sentence_list) < 4:
    return []

    # Correct sentences
  for sentence_idx in range(len(sentence_list) - 1):
    if len(range(sentence_idx + 1, len(sentence_list))) < 1:
      continue  # 샘플링할 개수가 부족하면 패스
    i_list = sample(range(sentence_idx + 1, len(sentence_list)), 1)
    cor_group_list = [[sentence_list[sentence_idx], sentence_list[i], 1] for i in i_list]
    sent_group.append(cor_group_list)

  # Incorrect sentences
  for sentence_idx in range(len(sentence_list) - 1):
    random_list = []

    valid_range = list(range(row_count + 1, len(context_df)))
    if len(valid_range) < 1:
      r_list = sample(valid_range, len(valid_range))  # 가능한 만큼만 샘플링
    else:
      r_list = sample(valid_range, 1)

    for r in r_list:
      random_context = context_df.shift(-r)[0]

      # random_context가 리스트가 아니거나 비어 있는 경우 건너뜀
      if not isinstance(random_context, list) or len(random_context) == 0:
          continue

      i = sample(range(len(random_context)), 1)[0]
      random_list.append(random_context[i])

    if len(random_list) > 0:  # 빈 리스트 방지
      wro_group_list = [[sentence_list[sentence_idx], random_sentence, 0] for random_sentence in random_list]
      sent_group.append(wro_group_list)

    row_count += 1  # row_count 증가

  return sent_group

In [11]:
labeling = context_df2.progress_apply(select_sentence)

100%|██████████| 24011/24011 [01:04<00:00, 370.65it/s] 


In [12]:
def dim_shrink(row):
  try:
    arr = np.array(row)  # 오류 발생 가능 부분
    a, b, c = arr.shape  # a = 뽑은 문장 개수*2, b = correct_sent, c = wrong_sent
    new_arr = arr.reshape(a * b, c)
    return new_arr.tolist()
  except Exception as e:
    return None  # 오류 발생 시 None 반환

In [13]:
shrinked_labeling = labeling.progress_apply(dim_shrink)

100%|██████████| 24011/24011 [00:04<00:00, 5299.69it/s]


In [14]:
train_dict = {'sent1':[],
              'sent2':[],
              'label':[]}
for row in tqdm(shrinked_labeling, ascii = False, desc = '딕셔너리 형성'):
  if row == None:
    continue
  for data in row:
    train_dict['sent1'].append(data[0])
    train_dict['sent2'].append(data[1])
    train_dict['label'].append(data[2])
train_df = pd.DataFrame(train_dict)

딕셔너리 형성: 100%|██████████| 24011/24011 [00:00<00:00, 53101.44it/s]


In [15]:
directory = "/content/drive/MyDrive/Colab Notebooks/비타민 프로젝트/NLP/train_data"
train_csv = os.path.join(directory, "train.csv")
train_df.to_csv(train_csv, index=False, encoding='utf-8')
print(f"Train CSV results saved to: {train_csv}")

Train CSV results saved to: /content/drive/MyDrive/Colab Notebooks/비타민 프로젝트/NLP/train_data/train.csv


In [16]:
directory = "/content/drive/MyDrive/Colab Notebooks/비타민 프로젝트/NLP/train_data"
train_json = os.path.join(directory, "train.json")
train_df.to_json(train_json, orient='records', force_ascii=False, indent=4)
print(f"Train JSON results saved to: {train_json}")

Train JSON results saved to: /content/drive/MyDrive/Colab Notebooks/비타민 프로젝트/NLP/train_data/train.json


## 학습 진행

In [1]:
from transformers import BertTokenizer, BertForNextSentencePrediction, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
import pandas as pd
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast

In [2]:
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForNextSentencePrediction.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForNextSentencePrediction were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df = pd.read_csv('train_data.csv')  # CSV 파일을 데이터셋으로 로드
sampled_df = df.sample(frac=0.75, random_state=42)

# 토크나이저 적용 함수
class NSPDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sent1 = self.data.iloc[idx]['sent1']
        sent2 = self.data.iloc[idx]['sent2']
        label = self.data.iloc[idx]['label']

        # KoBERT 토큰화
        tokens = self.tokenizer(sent1, sent2, return_tensors="pt", padding="max_length", truncation=True, max_length=128)

        return {
            'input_ids': tokens['input_ids'].squeeze(0),
            'attention_mask': tokens['attention_mask'].squeeze(0),
            'token_type_ids': tokens['token_type_ids'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 데이터셋 생성
dataset = NSPDataset(sampled_df, tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [7]:
# CUDA 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 옵티마이저 및 학습률 스케줄러 설정
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3
gradient_accumulation_steps = 4  # 그래디언트 누적 (큰 배치 효과)
scaler = GradScaler()  # AMP 활용
total_steps = len(dataloader) * epochs // gradient_accumulation_steps

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=total_steps // 10, num_training_steps=total_steps)

# 학습 루프
for epoch in range(epochs):
    model.train()
    loop = tqdm(dataloader, leave=True, desc=f"Epoch {epoch+1}")
    total_loss = 0

    optimizer.zero_grad()
    for step, batch in enumerate(loop):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["labels"].to(device)

        # Mixed Precision (AMP) 적용
        with autocast():
            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
            loss = outputs.loss / gradient_accumulation_steps  # 그래디언트 누적 고려

        # Backpropagation
        scaler.scale(loss).backward()
        total_loss += loss.item() * gradient_accumulation_steps

        # Gradient Accumulation 적용
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(dataloader):
            scaler.unscale_(optimizer)  # AMP 사용 시 그래디언트 스케일링 해제
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient Clipping
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()

        # 진행률 표시
        loop.set_postfix(loss=loss.item() * gradient_accumulation_steps)

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} 완료 | 평균 Loss: {avg_loss:.4f}")


C:\Users\leopp\anaconda3\envs\colab_env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\leopp\AppData\Local\Temp\ipykernel_23004\3879016690.py:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # AMP 활용
Epoch 1:   0%|                                                                               | 0/17434 [00:00<?, ?it/s]C:\Users\leopp\AppData\Local\Temp\ipykernel_23004\3879016690.py:28: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1:   3%|█▍                                                    | 454/17434 [19:16<13:27:29,  2.85s/it, loss=0.1

Epoch 1 완료 | 평균 Loss: 0.2441


Epoch 2:   6%|███▏                                                 | 1032/17434 [49:37<13:22:00,  2.93s/it, loss=0.231]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 2:   9%|████▌                                              | 1539/17434 [1:13:32<12:15:35,  2.78s/it, loss=0.409]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. 

Epoch 2 완료 | 평균 Loss: 0.2341


Epoch 3:   2%|▊                                                     | 271/17434 [12:41<13:09:39,  2.76s/it, loss=0.076]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 3:  19%|█████████▎                                        | 3255/17434 [2:33:38<10:53:54,  2.77s/it, loss=0.0992]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. 

Epoch 3 완료 | 평균 Loss: 0.2200


In [8]:
model.save_pretrained("./kobert_nsp_finetuned")
tokenizer.save_pretrained("./kobert_nsp_finetuned")

('./kobert_nsp_finetuned\\tokenizer_config.json',
 './kobert_nsp_finetuned\\special_tokens_map.json',
 './kobert_nsp_finetuned\\vocab.txt',
 './kobert_nsp_finetuned\\added_tokens.json')